<h1><center> Training a prompt model

# 1- Dataset

In [1]:

import numpy as np
import rasterio
import os 
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import os
import torch
import rasterio
import numpy as np
from torch.utils.data import Dataset,  DataLoader
from PIL import Image
from segment_anything import SamPredictor, sam_model_registry
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch
import os
import torch.nn.functional as F
import numpy as np
import tqdm
from transformers import SamModel


/Users/emmagauillard/Documents/MVA/Remote_sensing/projet/projet_remote/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_type = "vit_h"
path_to_checkpoint = '../checkpoint/sam_vit_h_4b8939.pth'

# Loading the model
sam_model = sam_model_registry[model_type](checkpoint=path_to_checkpoint)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
data_dir = "../split"

train_img_dir = os.path.join(data_dir, 'train', 'img')
test_img_dir = os.path.join(data_dir, 'val', 'img')
train_msk_dir = os.path.join(data_dir, 'train', 'msk')
test_msk_dir = os.path.join(data_dir, 'val', 'msk')

In [5]:
def simple_equalization_8bit(im, percentiles=5):
    ''' im is a numpy array
        returns a numpy array
    '''
    out = np.zeros_like(im)
    # faire l'equalization par channel
    def equalize(im_channel):
        v_min, v_max = np.percentile(im_channel,percentiles),np.percentile(im_channel, 100 - percentiles)

        # Clip the image to the percentile values
        im_clipped = np.clip(im_channel, v_min, v_max)

        # Scale the image to the 0-255 range
        im_scaled = np.round((im_clipped - v_min) / (v_max - v_min))
        return im_scaled.astype(np.uint8)
    
    for channel in range(im.shape[0]):
        out[channel,:,:] = equalize(im[channel,:,:])
    
    return out

In [10]:
class S1S2Dataset(Dataset):
    def __init__(self, img_folder, mask_folder, processor, transform=None, target_transform=None,):
        self.img_folder = img_folder
        self.mask_folder = mask_folder
        self.transform = transform
        self.target_transform = target_transform
        self.img_filenames = [f for f in os.listdir(img_folder) if f.endswith('.tif') and "img" in f]
        self.processor = processor

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        filename = self.img_filenames[idx]
        img_path = os.path.join(self.img_folder, filename)
        
        # image
        with rasterio.open(img_path) as src:
            image = src.read().astype(np.float32)[0:3, :, :]
            image = simple_equalization_8bit(image, percentiles=5) 
            image = torch.from_numpy(image) # shape (C, H, W)
            
            image = self.processor(image, return_tensors="pt")
            # remove batch dimension
            image = {k: v.squeeze(0) for k, v in image.items()}

        # masque
        mask_filename = filename.replace("img", "msk")
        mask_path = os.path.join(self.mask_folder, mask_filename)
        with rasterio.open(mask_path) as src:
            mask = src.read()[0].astype(np.float32)
            mask = torch.from_numpy(mask)
            mask = mask.unsqueeze(0)  # Ajouter une dimension de canal (C, H, W)

        # if self.transform:
        #     image = self.transform(image)
        #     mask = self.target_transform(mask)

        return image, mask

transform= transforms.Compose([
    transforms.Resize((1024, 1024)), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_target = transforms.Compose([transforms.Resize((224, 224))])


In [11]:
from transformers import SamProcessor

processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

dataset = S1S2Dataset(train_img_dir, train_msk_dir, transform=transform, processor=processor)

In [12]:
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [13]:
print("train_loader:", len(train_loader))
print(next(iter(train_loader))[0]["pixel_values"].shape)

/Users/emmagauillard/Documents/MVA/Remote_sensing/projet/projet_remote/env/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


train_loader: 3290
torch.Size([4, 3, 1024, 1024])


# 2- SAM

In [14]:
# Fine-tuning the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
model = SamModel.from_pretrained("facebook/sam-vit-base").to(device)
# make sure we only compute gradients for mask decoder
for name, param in model.named_parameters():
  if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
    param.requires_grad_(False)

In [ ]:
model = model.to(device)

# Define loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.mask_decoder.parameters(), lr=0.001, momentum=0.9)

model.train()
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in tqdm.tqdm(enumerate(train_loader, 0)):
        inputs, labels = data
        labels = labels.float().to(device)

        outputs = model(pixel_values=inputs["pixel_values"].to(device), multimask_output=False)
        
        predicted_masks = outputs.pred_masks.squeeze(1)
        print(' predicted_masks',predicted_masks.shape)
        print('labels', labels.shape)
        
        loss = criterion(predicted_masks, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(loss.item())
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

# torch.save(model.state_dict(), PATH)

print('Finished Training')